In [2]:
import biosppy.signals.resp as resp
import numpy as np
import csv as csv
import json    
import matplotlib.pyplot as plt


def calc_resp_intervals(data, last_breath = False):
    processed_data = resp.resp(signal=data, sampling_rate=200, show=False)
    filtered_signal = processed_data[1]
    inst_resp_rate = processed_data[4]
    
    signal_diff = np.diff(filtered_signal)
    signal_signum = signal_diff > 0
    
    resp_changes = np.append(np.where(signal_signum[:-1] != signal_signum[1:])[0], [len(signal_signum) - 1])
    
    if not last_breath:
        resp_intervals = np.append([0], resp_changes)
    
        interval_lengths = np.diff(resp_intervals)
    
        interval_breathe_in = [signal_signum[i] for i in resp_changes]
        
        return interval_lengths, interval_breathe_in
    
    else:
        last_interval = resp_changes[-1] - resp_changes[-2]
        
        return last_interval, signal_signum[resp_changes[-1]]
        

In [6]:
# resp_intervals: length of inhalation/exhalation in samples
# is_inhalation: True -> inhale, False: exhale
def resp_features(resp_intervals, is_inhalation, samplingRate):
    
    # calculate the average breath length in s
    breath_lengths = []
    i = 0

    while i < len(resp_intervals):
        if not is_inhalation[i] or i == len(resp_intervals) - 1 :
            breath_lengths.append(resp_intervals[i])
            i = i + 1
        else:
            breath_lengths.append(resp_intervals[i] + resp_intervals[i + 1])
            i = i + 2
    
    avg_breath = np.average(breath_lengths) / samplingRate
    
    # calculate the inhalation/exhalation ratio
    breathe_in_len = 0
    breathe_out_len = 0
    
    for i in range(len(interval_lengths)):
        if interval_breathe_in[i]:
            breathe_in_len += interval_lengths[i]
        else:
            breathe_out_len += interval_lengths[i]
    
    in_out_ratio = breathe_in_len/breathe_out_len
    
    features = {'breath_avg_len': avg_breath, 'inhale_duration': breathe_in_len / samplingRate, 
                'exhale_duration': breathe_out_len / samplingRate, 'inhale_exhale_ratio': in_out_ratio}
    
    return features
    

In [4]:
fname = '../riot_rip_acquisition/ak/opensignals_192.168.1.1008888_2020-02-05_11-54-42.txt'
data = np.genfromtxt(fname)
col_num  = 5
ripdata = data[:,col_num]

In [7]:
interval_lengths, interval_breathe_in = calc_resp_intervals(ripdata)

resp_features = resp_features(interval_lengths, interval_breathe_in, 200)

In [8]:
resp_features

{'breath_avg_len': 3.488985507246377,
 'inhale_duration': 124.875,
 'exhale_duration': 115.865,
 'inhale_exhale_ratio': 1.0777629137358131}